In [ ]:
!pip install boto3 progressbar2 sfdmap GPUtil

In [ ]:
pip install typing-extensions --upgrade

In [ ]:
!pip install "holoviews[recommended]"

In [ ]:
!pip install plotly

In [ ]:
# imports
import pandas as pd
import numpy as np
import os
import sys
import pickle
from matplotlib import pyplot as plt
import matplotlib
import boto3

# random seed
seed = 42
np.random.seed(seed)

# local files paths
local_home_dir_path = os.path.expanduser("~")
local_work_dir_path = os.path.join(local_home_dir_path, 'git')
local_code_dir_path = os.path.join(local_work_dir_path , 'code')

# S3 file paths
endpoint_url = 'https://s3.nautilus.optiputer.net'
bucket_name = 'tau-astro'
prefix = 'almogh'
s3_work_dir_path = os.path.join(prefix, 'workdir3')
s3_saves_dir_path = os.path.join(s3_work_dir_path , 'model_saves')
s3_data_dir_path = os.path.join(s3_work_dir_path , 'data')
s3_data_ver_dir_path = os.path.join(s3_data_dir_path,'HighSNR_12K_V1')

s3_client = boto3.client("s3", endpoint_url=endpoint_url)

# adding code folder to path
sys.path.insert(1, local_code_dir_path)
from s3 import to_s3_npy, to_s3_pkl, from_s3_npy, from_s3_pkl, to_s3_fig

# Load all data

In [ ]:
save_dir_name = 'simple___2021_11_27___22_09_00___standard_RF_max_depth_10'
s3_rf_save_dir_path = os.path.join(s3_saves_dir_path, 'RF', save_dir_name)

In [ ]:
print('Loading saved data')
I_slice = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'I_slice.npy'))
I_train = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'I_train.npy'))
X_train_real = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'X.npy'))
X_leaves_train_real = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'X_leaves.npy'))
Y_hat_train_real = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'Y_hat.npy'))
weird_scores = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'weird_scores.npy'))
sne = from_s3_npy(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'tsne.npy'))

print('Loading wavelength grid and dataframe')
wl_grid = from_s3_npy(s3_client, bucket_name, os.path.join(s3_data_dir_path, 'wl_grid.npy'))
gs = from_s3_pkl(s3_client, bucket_name, os.path.join(s3_data_ver_dir_path, 'gs.pkl'))
I_real_train = I_train[I_train<len(I_slice)]
snr = gs.snMedian.iloc[I_slice[I_real_train]]

print('Loading random forest')
from CustomRandomForest import CustomRandomForest
rf = CustomRandomForest.load_s3(s3_client, bucket_name, os.path.join(s3_rf_save_dir_path, 'crf.pkl'))

print('done.')

# Plot

In [ ]:
import holoviews as hv
from holoviews import opts
from holoviews.streams import Selection1D
from bokeh.models import HoverTool
from scipy import stats
import panel as pn
hv.extension('bokeh')

In [ ]:
# creading the dataframe
df = pd.DataFrame(sne, columns=['feature_1', 'feature_2'])
df['score'] = weird_scores
df['snr'] = snr
df['index'] = np.arange(len(df))

In [ ]:
def points_dmap_callable(color_src):
    """
    The callable function for the points DynamicMap.
    """
    points = hv.Points(df).opts(color=color_src, cmap='jet')
    points.opts(tools=['tap','box_select','lasso_select'])
    points.opts(selection_alpha=0.4, nonselection_alpha=0.1)
    return points

In [ ]:
from datetime import datetime

def spectra_dmap_callable(index):
    """
    The callable function for the spectra DynamicMap.
    """
    with open('/home/jovyan/debug.txt','w') as f:
        f.write('in spectra_dmap_callable - '+datetime.now().strftime("%d/%m/%Y %H:%M:%S")+'\n')
        if len(index)==0:
            f.write('len==0\n')
            # No Selection
            x = np.zeros(shape=wl_grid.shape)
            w = wl_grid
            label = 'No Selection'
        else:
            f.write('len!=0\n')
            x = np.nanmean(X_train_real[index], axis=0)
            x_valid = ~np.isnan(x)
            x = x[x_valid]
            w = wl_grid[x_valid]
            if len(index)==1:
                f.write('len==1\n')
                # a single point - plotting the outlier feature importance
                label = 'index=%s, snr=%f, score=%f' % (index[0], snr[index[0]], weird_scores[index[0]])
            else:
                f.write('len>1\n')
                # Multiple points - plotting the cluster feature importance
                label = '%d points selected - plotting the average' % len(index)

        flux = hv.Curve((w,x), kdims=['w'],vdims=['flux']).opts(color='black').opts(norm=dict(framewise=True)).opts(tools=['hover'])
    
    return flux.relabel(label).opts(width=800, height=300, show_grid=True)

In [ ]:
def examples_dmap_callable(index):
    """
    The callable function for the examples DynamicMap.
    """
    spectra_dict = {i:hv.Curve((wl_grid,X_train_real[ind])).relabel('index=%s, snr=%f, score=%f' % (ind, snr[ind], weird_scores[ind])).opts(width=800, height=300, show_grid=True) for i,ind in enumerate(index)}

    return hv.HoloMap(spectra_dict, kdims='index')

In [ ]:
# Creating the T-SNE Points DynamicMap (Dynamic because of 2 color sources - score and snr)
points_dmap = hv.DynamicMap(points_dmap_callable, kdims='ColorSource').redim.values(ColorSource=['score','snr'])
points_dmap.opts(framewise=True, width=800, height=500, colorbar=True)
selection = Selection1D(source=points_dmap) # creating a selection from the points

# Creating the spectra DynamicMap - to load the spectra according to the selection from the selection of the T-SNE points.
spectra_dmap = hv.DynamicMap(spectra_dmap_callable, kdims=[], streams=[selection])
spectra_dmap.opts(norm=dict(framewise=True))

# Building the layout full layout
layout = (points_dmap + spectra_dmap).opts(merge_tools=False)
layout.cols(1)

pn.pane.HoloViews(layout, widget_location="top")

In [ ]:
points_dmap

# Manual plot

In [ ]:
from matplotlib import pyplot as plt
I_weird = np.argsort(weird_scores)
i = 0
print(weird_scores[I_weird[i]])
fig,ax = plt.subplots(figsize=(12,8))
ax.plot(wl_grid, X_train_real[I_weird[i],:])
ax.grid()

# Other stuff

In [ ]:
gs = data['gs'].iloc[I_train[I_train<len(data['gs'])]]

In [ ]:
g = gs.iloc[6899]
from NAURF import getUrl
from astropy.io import fits
url = getUrl(g)
# Get data from .fits file (download from url)
hdulist = fits.open(url, memmap=False, cache=False)
data = hdulist[1].data
specobjid = int(hdulist[2].data['specobjid'].item())
hdulist.close()

# Make sure the file matches the galaxy desired
assert str(specobjid) == g['specobjid'], 'Files do not match galaxies dataframe'

# Get flux values and wavelengths
y = data['flux']
x = np.array(10 ** data['loglam'], dtype=float)
dy = data['ivar']
hv.Curve((x,y)).opts(width=800, height=300, show_grid=True)

# Auxilary Plots

In [ ]:
holomap = examples_dmap_callable(np.load(os.path.join(save_dir_path, 'cluster.npy'))).opts(width=800, height=300, show_grid=True)
holomap_panel = pn.panel(holomap)
plot = holomap_panel[0]
widgets = holomap_panel[1]
new_layout_panel = pn.Column(plot, widgets)
new_layout_panel.servable()

# Debugging

In [ ]:
# plotting the mean spectra of the dataset
hv.Curve((wl_grid, np.nanmean(X_train_real, axis=0))).opts(width=800, height=300, show_grid=True)

In [ ]:
# plotting the importances of the saved cluster
index = np.load(os.path.join(save_dir_path, 'cluster.npy')) # loading the saved indices
importance = create_wavelengths_importance(index, filt=False, norm=False)
hv.Curve((wl_grid, importance)).opts(width=800, height=300, show_grid=True).opts(tools=['hover'])

In [ ]:
# plotting the importances of the saved cluster
index = np.load(os.path.join(save_dir_path, 'cluster.npy')) # loading the saved indices
importance = create_wavelengths_importance(index, filt=False, norm=False)
hv.Curve((wl_grid, importance)).opts(width=800, height=300, show_grid=True).opts(tools=['hover'])

In [ ]:
# plotting the importances of the saved cluster
index = np.load(os.path.join(save_dir_path, 'cluster.npy')) # loading the saved indices
importance = create_wavelengths_importance(index, filt=False, norm=False)
hv.Curve((wl_grid, importance)).opts(width=800, height=300, show_grid=True).opts(tools=['hover'])

In [ ]:
# plotting the marginal distribution 
wl = 5658
wl_i = np.where(wl_grid==wl)[0][0]
x_cluster = X_train_real[index,wl_i]
x_real = X_train[y_train==1,wl_i]
x_synth = X_train[y_train==2,wl_i]

grid = np.linspace(0,3,20)
cluster_freq, cluster_edges = np.histogram(x_cluster, grid, density=True)
real_freq, real_edges = np.histogram(x_real[~np.isnan(x_real)], grid, density=True)
synth_freq, synth_edges = np.histogram(x_synth[~np.isnan(x_synth)], grid, density=True)
cluster_centers = (cluster_edges[:-1]+cluster_edges[1:])/2
real_centers = (real_edges[:-1]+real_edges[1:])/2
synth_centers = (synth_edges[:-1]+synth_edges[1:])/2
comp = hv.Curve((cluster_centers, cluster_freq),label='cluster')*hv.Curve((real_centers, real_freq), label='real')*hv.Curve((synth_centers, synth_freq), label='synth')
comp.opts(width=800, height=300, show_grid=True)

# Other code...

In [ ]:
# Creating the decision paths dictionary
from progressbar import progressbar
decision_paths = dict()
for k in progressbar(range(len(rf.estimators_))):
    i_real = np.where(Y_hat_train_real[:,k]==1)[0] # indices of all samples which the k-th tree classified as "real"
    i_sample,i_node = rf.estimators_[k].tree_.decision_path(X_train_real[i_real,rf.tree_i_start[k]:rf.tree_i_end[k]].astype(np.float32)).nonzero()
    i_feature = rf.tree_i_start[k]+rf.estimators_[k].tree_.feature[i_node]
    for i in range(len(i_real)):
        temp_feature = i_feature[i_sample==i]
        decision_paths[(i_real[i],k)] = temp_feature[:-1] # the last node is a leaf (not a decision node)
        
# saving it
import pickle
with open(decision_paths_path,'wb') as f:
    pickle.dump(decision_paths, f)
    
# decision_paths_2 = [[decision_paths[(i,k)] if (i,k) in decision_paths else None for k in range(Y_hat_train_real.shape[1])] for i in range(Y_hat_train_real.shape[0])]